In [1]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import json
import os
import random
import glob
# from detectron_pro import detectron_mask_img,detectron_mask_img_composite
import shutil
import cv2

import random

In [2]:

# to get location that stickimg will sticked on jpg_dir center or random
def center_location(jpg_dir,stickimg_dir):
    im = np.array(Image.open(jpg_dir), dtype=np.uint8)
    x_center,y_center = im.shape[1]/2,im.shape[0]/2
    im_stick = np.array(Image.open(stickimg_dir), dtype=np.uint8)
    im_stick_shape = im_stick.shape
    bd_box_x,bd_box_y = x_center-(im_stick_shape[1]/2),y_center-(im_stick_shape[0]/2)
    bd_box_length,bd_box_height =im_stick_shape[1],im_stick_shape[0]
    return bd_box_x,bd_box_y,bd_box_length,bd_box_height

def random_location(jpg_dir,stickimg_dir):
    im = np.array(Image.open(jpg_dir), dtype=np.uint8)
    # x boundary
    #rangeX = 400
    #x_left_bound,x_right_bound = rangeX, im.shape[1]-rangeX
    x_left_bound,x_right_bound = 128,512
    x_center = random.randrange(x_left_bound, x_right_bound)
    seq = [320,280,240,220,200,190,180,175]
    y_center = random.choice(seq)
    i_shape1 = [240,120]
    i_shape2 = [160,80]
    i_shape3 = [128,64]
    i_shape4 = [108,54]
    i_shape5 = [96,48]
    i_shape6 = [72,36]
    i_shape7 = [60,30]
    i_shape8 = [56,28]
    dict = {320:i_shape1,280:i_shape2 ,240:i_shape3,220:i_shape4,200:i_shape5,190:i_shape6,180:i_shape7,175:i_shape8}
    image_shape = dict[y_center]
    bd_box_x,bd_box_y = x_center-(image_shape[1]/2),y_center-(image_shape[0]/2)
    bd_box_length,bd_box_height =image_shape[1],image_shape[0]
    return bd_box_x,bd_box_y,bd_box_length,bd_box_height


#h_list = [200,220,240,296,352] # from top to buttom (center of people image)
#h_scale = [(48,96),(54,108),(64,128),(80,160),(120,240)] # top might be smaller

In [8]:
#it can generate json file defined by your street_jpg and people_jpg 

def create_json_file(jpg_dir,street_json,stickimg_dir,results_dir,function='center'):
    if function == 'center':
        bd_box_x,bd_box_y,bd_box_length,bd_box_height = center_location(jpg_dir,stickimg_dir)
    elif function == 'random':
        bd_box_x,bd_box_y,bd_box_length,bd_box_height = random_location(jpg_dir,stickimg_dir)
        
    input_file = open (street_json)
    json_array = json.load(input_file)
    
    data = []
    data.append({
    'end':0,
    'hide':0,
    'id':0,
    'init':0,
    'lbl':"pasted_person",
    'lock':0,
    'occl':0,
    'pos':[
    bd_box_x,
    bd_box_y,
    bd_box_length,
    bd_box_height],    
    'posv':[
    0,
    0,
    0,
    0],
    'str':0
                })
    
    if json_array != []:
        for item in json_array:
            data.append(item)
            
    with open(results_dir, 'w') as outfile:
        json.dump(data, outfile)

# Start create

In [9]:
origin_data_dir = '/root/notebooks/final/caltech_pedestrian_refine_no'

In [10]:
img_data = glob.glob(origin_data_dir+'/street/*.jpg', recursive=True)

json_data = glob.glob(origin_data_dir+'/street_json/*.json', recursive=True)

In [11]:
img_data[:10]

['/root/notebooks/final/caltech_pedestrian_refine_no/street/013258.jpg',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street/013259.jpg',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street/013260.jpg',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street/013261.jpg',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street/013262.jpg',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street/013263.jpg',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street/013264.jpg',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street/013265.jpg',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street/013266.jpg',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street/013267.jpg']

In [12]:
json_data[:10]

['/root/notebooks/final/caltech_pedestrian_refine_no/street_json/013258.json',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street_json/013259.json',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street_json/013260.json',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street_json/013261.json',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street_json/013262.json',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street_json/013263.json',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street_json/013264.json',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street_json/013265.json',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street_json/013266.json',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street_json/013267.json']

In [13]:
# Image read dir
street_dir = '/root/notebooks/0858611-2/final_project/caltech_pedestrian_extractor/video_extractor/*'

people_dir = '/root/notebooks/final/market_mask2'

# Image save dir
save_dir = '/root/notebooks/final/result_dataset_9'

num_imgs = 10000

In [14]:
# Check dir folder exit
# If not, create one
if os.path.exists(save_dir) == False:
    os.makedirs(save_dir)

for s in ['people', 'mask', 'street', 'street_json','json']:
    if os.path.exists(os.path.join(save_dir, s)) == False:
        os.makedirs(os.path.join(save_dir, s))

In [15]:
#street_imgs = glob.glob(street_dir+'/**/*.jpg', recursive=True)
street_imgs = img_data

#street_imgs = random.shuffle(random.sample(street_imgs, 5000))
street_imgs = random.sample(street_imgs, num_imgs)

random.shuffle(street_imgs)

In [16]:
#people_imgs_ = glob.glob(people_dir+'/people/*.jpg', recursive=True)

mask_imgs_ = glob.glob(people_dir+'/market_mask_refine_6467/*.jpg', recursive=True)

people_imgs_ = [i.replace('market_mask_refine_6467', 'people') for i in mask_imgs_]

#mask_imgs_ = [i.replace('people', 'mask') for i in people_imgs_]

#people_imgs = random.sample(people_imgs, num_imgs)

r_t_imgs = []
r_m_imgs = []

In [19]:
mask_imgs_[:10]

['/root/notebooks/final/market_mask2/market_mask_refine_6467/003301.jpg',
 '/root/notebooks/final/market_mask2/market_mask_refine_6467/013112.jpg',
 '/root/notebooks/final/market_mask2/market_mask_refine_6467/001516.jpg',
 '/root/notebooks/final/market_mask2/market_mask_refine_6467/011705.jpg',
 '/root/notebooks/final/market_mask2/market_mask_refine_6467/011063.jpg',
 '/root/notebooks/final/market_mask2/market_mask_refine_6467/001270.jpg',
 '/root/notebooks/final/market_mask2/market_mask_refine_6467/002779.jpg',
 '/root/notebooks/final/market_mask2/market_mask_refine_6467/013674.jpg',
 '/root/notebooks/final/market_mask2/market_mask_refine_6467/013660.jpg',
 '/root/notebooks/final/market_mask2/market_mask_refine_6467/010369.jpg']

In [20]:
# for i in range(len(people_imgs_)):
#     if (i%1000==0):
#         print("Process (",i,"/",len(people_imgs_),")  ","{:.2f}".format(100*i/len(people_imgs_))," %")
#     r_t_imgs.append(cv2.imread(people_imgs_[i]))
#     r_m_imgs.append(cv2.imread(mask_imgs_[i]))

In [21]:
#people_imgs = [people_imgs_[random.randint(0,len(people_imgs_)-1)] for i in range(num_imgs)]


#random.shuffle(people_imgs)

In [23]:
street_imgs[:10]

['/root/notebooks/final/caltech_pedestrian_refine_no/street/094801.jpg',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street/021181.jpg',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street/059220.jpg',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street/022438.jpg',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street/072103.jpg',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street/057311.jpg',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street/055315.jpg',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street/002845.jpg',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street/098081.jpg',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street/083565.jpg']

In [24]:
for i in range(num_imgs):
    
    if (i%100==0):
        print("Process (",i,"/",num_imgs,")  ","{:.2f}".format(100*i/num_imgs)," %")
        
    people_pick = random.randint(0,len(people_imgs_)-1)
    
    # create mask and save
    #try:
    #    mask_img = detectron_mask_img(people_imgs[i],(64,128))
    #    mask_img = Image.fromarray(mask_img)
    #except Exception as e:
    #    print("Skip image :",i)
    #    continue
    
    mask_img = mask_imgs_[people_pick]
        
    #mask_img.save(save_dir+'/mask/'+str('{0:06}'.format(i))+'.jpg')
    
    shutil.copyfile(mask_img, save_dir+'/mask/'+str('{0:06}'.format(i))+'.jpg')
    
    # save street img
    street_img = cv2.imread(street_imgs[i])
    street_img = cv2.resize(street_img,(640,480))
    cv2.imwrite(save_dir+'/street/'+str('{0:06}'.format(i))+'.jpg', street_img)
    
    ################################################################
    img_path = street_imgs[i]
    json_dir = img_path.replace('street', 'street_json')
    json_dir = json_dir.replace('jpg', 'json')
    shutil.copyfile(json_dir, save_dir+'/street_json/'+str('{0:06}'.format(i))+'.json')
    ################################################################
    
    # save poeple img
    #people_img = cv2.imread(people_imgs[i])
    people_img = people_imgs_[people_pick]
    #people_img = cv2.resize(people_img,(64,128))
    #cv2.imwrite(save_dir+'/people/'+str('{0:06}'.format(i))+'.jpg', people_img)
    shutil.copyfile(people_img, save_dir+'/people/'+str('{0:06}'.format(i))+'.jpg')
    
    # create json file and save
    create_json_file(save_dir+'/street/'+str('{0:06}'.format(i))+'.jpg',
                     save_dir+'/street_json/'+str('{0:06}'.format(i))+'.json',
                     save_dir+'/people/'+str('{0:06}'.format(i))+'.jpg',
                     save_dir+'/json/'+str('{0:06}'.format(i))+'.json',
                     function="random")

Process ( 0 / 10000 )   0.00  %
Process ( 100 / 10000 )   1.00  %
Process ( 200 / 10000 )   2.00  %
Process ( 300 / 10000 )   3.00  %
Process ( 400 / 10000 )   4.00  %
Process ( 500 / 10000 )   5.00  %
Process ( 600 / 10000 )   6.00  %
Process ( 700 / 10000 )   7.00  %
Process ( 800 / 10000 )   8.00  %
Process ( 900 / 10000 )   9.00  %
Process ( 1000 / 10000 )   10.00  %
Process ( 1100 / 10000 )   11.00  %
Process ( 1200 / 10000 )   12.00  %
Process ( 1300 / 10000 )   13.00  %
Process ( 1400 / 10000 )   14.00  %
Process ( 1500 / 10000 )   15.00  %
Process ( 1600 / 10000 )   16.00  %
Process ( 1700 / 10000 )   17.00  %
Process ( 1800 / 10000 )   18.00  %
Process ( 1900 / 10000 )   19.00  %
Process ( 2000 / 10000 )   20.00  %
Process ( 2100 / 10000 )   21.00  %
Process ( 2200 / 10000 )   22.00  %
Process ( 2300 / 10000 )   23.00  %
Process ( 2400 / 10000 )   24.00  %
Process ( 2500 / 10000 )   25.00  %
Process ( 2600 / 10000 )   26.00  %
Process ( 2700 / 10000 )   27.00  %
Process ( 2800